In [1]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.sparse as sparse
# import skimage
# import skimage.filters as filters
import skimage
from scipy import ndimage, signal
from scipy.interpolate import interp1d
# from skimage import color, util
# from skimage.util import img_as_uint
from sksparse.cholmod import cholesky

# # Closed-Form Scribble UI
# import tkinter
# from tkinter import ttk

from tqdm.auto import tqdm
import numba as nb

/home/jacky/Documents/part-iii/III Notes/src/L352-AGIP/project/l352-image-matting/matting/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_name = "GT01.png"
image_path = os.path.join("../datasets", "input_training_lowres", image_name)
im = skimage.io.imread(image_path)

In [3]:
H, W, C = im.shape  # dim 0: rows. dim 1: cols. dim 2: colours.

# UI Trial. 
You know what. Just manual. This is taking too much time.

In [ ]:
# References:
# - https://pycad.co/how-to-draw-on-an-image/
# - https://tkdocs.com/tutorial/canvas.html
root = tkinter.Tk()
root.title("Scribble")
root.resizable(False, False)
# root.geometry(f"{W}x{H}")  # in tkinter, x is width, y is height
mainframe = ttk.Frame(root, padding="3 3 12 120")
mainframe.grid(column=0, row=0, sticky="NSEW")

# canvas = tkinter.Canvas(root, bg="black")
# canvas.pack(anchor="nw", fill="both", expand=True)

class Sketchpad(tkinter.Canvas):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.bind("<Button-1>", self.save_position)
        self.bind("<ButtonRelease-1>", self.letgo)
        self.bind("<B1-Motion>", self.add_line)
        self.last_x = None
        self.last_y = None
        self.ink_colour = "white"
        self.ink_colour_text = tkinter.StringVar(parent, "Drawing: Foreground")
        self.ink_size = tkinter.IntVar(parent, 10)
        # self.ink_size_text = tkinter.StringVar(parent, f"Ink Size: {self.ink_size}")

    def set_ink_colour(self, colour):
        if colour == "white":
            self.ink_colour_text.set("Drawing: Foreground")
        elif colour == "black":
            self.ink_colour_text.set("Drawing: Background")
        else:
            raise NotImplementedError
        self.ink_colour = colour

    # def set_ink_size(self,)

    def save_position(self, event):
        self.last_x, self.last_y = event.x, event.y

    def add_line(self, event):
        self.create_line(
            (self.last_x, self.last_y, event.x, event.y),
            fill=self.ink_colour,
            width=self.ink_size.get()
        )
        self.save_position(event)

    def letgo(self, _):
        all_segment_ids = self.find_all()
        all_segments = []
        for isegment in range(len(all_segment_ids)):
            all_segments.append(self.coords(isegment))
        print(all_segments)


# image_view = tkinter.Canvas(mainframe, height=H, width=W)
# image_view.create_image(0, 0, image=tkinter_image, anchor="nw")
# image_view.grid(row=0, column=0, columnspan=3, sticky="NW")

tkinter_image = tkinter.PhotoImage(master=root, file=image_path)
canvas = Sketchpad(mainframe, width=W, height=H)
canvas.create_image(0, 0, image=tkinter_image, anchor="nw")
canvas.grid(column=0, row=0, columnspan=3, sticky="NW")

ink_colour_text = ttk.Label(mainframe, textvariable=canvas.ink_colour_text)
ink_colour_text.grid(row=1, column=0)
foreground_button = ttk.Button(mainframe, text="Foreground", command=lambda: canvas.set_ink_colour("white"))
foreground_button.grid(row=1, column=1)
background_button = ttk.Button(mainframe, text="Background", command=lambda: canvas.set_ink_colour("black"))
background_button.grid(row=1, column=2)

ink_size_text = ttk.Label(mainframe, textvariable=canvas.ink_size)
ink_size_text.grid(row=2, column=0)
ink_size_slider = ttk.Scale(mainframe, from_=1, to=100, variable=canvas.ink_size, orient="horizontal")
ink_size_slider.grid(row=2, column=1, columnspan=2, sticky="EW")


# unknown_button = ttk.Button(mainframe, text='Okay', command=lambda: print("fuck"))



# canvas.create_image(0, 0, image=tkinter_image, anchor="nw")
# canvas.bind("<Button-1>", save_position)
# canvas.bind("<B1-Motion>", functools.partial(add_line, canvas))

# # ttk.Label(frm, text="Hello World!").grid(column=0, row=0)
# ttk.Button(frm, text="Quit", command=root.destroy).grid(column=1, row=0)
root.mainloop()

In [ ]:

# plt.axis('off')
skimage.io.imshow(im)
plt.show()

In [ ]:
im.shape

# Main

In [20]:
# im  # dim 0: rows. dim 1: cols. dim 2: colours.
# im = im.reshape(-1, 3)  # row-major traversal of image
# im

M = 3  # window size
assert M % 2 != 0, f"Window size M should be even, got {M}"
pad_amount = (M - 1) // 2
zero_padded_im = np.zeros((H + pad_amount * 2, W + pad_amount * 2, 3))
zero_padded_im[pad_amount:-pad_amount, pad_amount:-pad_amount] = im
zero_padded_im[:,:,0]

t = np.zeros((H + pad_amount * 2, W + pad_amount * 2))
t[pad_amount:-pad_amount, pad_amount:-pad_amount] = 1
unpadded_to_padded_index_map = [(x, y) for x, y in zip(*t.nonzero())]

neighbourhoods = []
mu_ks = np.zeros((H * W, 3))
Sigma_ks = np.zeros((H * W, 3, 3))

# k is centre of window
for k in tqdm(np.arange(H * W)):
    padded_x, padded_y = unpadded_to_padded_index_map[k]
    pixels = zero_padded_im[padded_x - pad_amount:padded_x + pad_amount + 1, padded_y - pad_amount:padded_y + pad_amount + 1]
    neighbourhoods.append(np.copy(pixels.reshape(-1, 3)))

import concurrent.futures
# @nb.njit#(parallel=True)
# def g():

def f(pixels):
    mu_k = np.mean(pixels, axis=0)
    return mu_k
    # mu_ks[k] = mu_k
    # Sigma_k = np.cov(pixels, rowvar=False, bias=True)  # bias True to divide by M^2 instead of (M^2 - 1)
    # Sigma_ks[k] = Sigma_k

with concurrent.futures.ThreadPoolExecutor() as executor:
    result = list(executor.map(f, neighbourhoods[:10]))

np.array(result)


100%|██████████| 397600/397600 [00:01<00:00, 254486.41it/s]


array([[ 47.22222222,  51.        ,  65.66666667],
       [ 70.77777778,  76.66666667,  99.22222222],
       [ 70.77777778,  77.33333333, 100.66666667],
       [ 71.22222222,  77.77777778, 102.        ],
       [ 71.88888889,  78.44444444, 103.        ],
       [ 72.22222222,  79.        , 103.88888889],
       [ 72.22222222,  79.66666667, 104.55555556],
       [ 72.22222222,  80.22222222, 105.33333333],
       [ 72.22222222,  80.66666667, 105.88888889],
       [ 72.44444444,  80.88888889, 106.55555556]])

In [13]:
mu_ks

array([[ 47.22222222,  51.        ,  65.66666667],
       [ 70.77777778,  76.66666667,  99.22222222],
       [ 70.77777778,  77.33333333, 100.66666667],
       ...,
       [ 85.33333333,  91.66666667, 108.22222222],
       [ 84.44444444,  90.55555556, 107.66666667],
       [ 56.        ,  60.        ,  71.66666667]])

In [14]:
Sigma_ks

array([[[2787.72839506, 3010.77777778, 3876.40740741],
        [3010.77777778, 3252.22222222, 4187.33333333],
        [3876.40740741, 4187.33333333, 5391.55555556]],

       [[2505.28395062, 2713.81481481, 3511.71604938],
        [2713.81481481, 2940.44444444, 3805.2962963 ],
        [3511.71604938, 3805.2962963 , 4925.72839506]],

       [[2505.28395062, 2737.51851852, 3563.48148148],
        [2737.51851852, 2991.55555556, 3893.77777778],
        [3563.48148148, 3893.77777778, 5071.11111111]],

       ...,

       [[3641.11111111, 3911.11111111, 4617.7037037 ],
        [3911.11111111, 4201.55555556, 4960.18518519],
        [4617.7037037 , 4960.18518519, 5857.50617284]],

       [[3568.24691358, 3825.75308642, 4547.81481481],
        [3825.75308642, 4103.13580247, 4876.96296296],
        [4547.81481481, 4876.96296296, 5798.        ]],

       [[3922.22222222, 4201.55555556, 5018.33333333],
        [4201.55555556, 4502.        , 5376.66666667],
        [5018.33333333, 5376.66666667, 642

In [50]:
t = np.arange(25).reshape(5,5)
[(a,b) for a,b in zip(*np.nonzero(t))]

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [7]:
np.arange(60).reshape((5,4,3))

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]],

       [[12, 13, 14],
        [15, 16, 17],
        [18, 19, 20],
        [21, 22, 23]],

       [[24, 25, 26],
        [27, 28, 29],
        [30, 31, 32],
        [33, 34, 35]],

       [[36, 37, 38],
        [39, 40, 41],
        [42, 43, 44],
        [45, 46, 47]],

       [[48, 49, 50],
        [51, 52, 53],
        [54, 55, 56],
        [57, 58, 59]]])

In [16]:
np.arange(60).reshape((5,4,3)).reshape((20, 3))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20],
       [21, 22, 23],
       [24, 25, 26],
       [27, 28, 29],
       [30, 31, 32],
       [33, 34, 35],
       [36, 37, 38],
       [39, 40, 41],
       [42, 43, 44],
       [45, 46, 47],
       [48, 49, 50],
       [51, 52, 53],
       [54, 55, 56],
       [57, 58, 59]])